In [1]:
import wandb
wandb.login()

/Users/mrinoyb2/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/Users/mrinoyb2/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to y

True

## Using Adam Optimizer

In [10]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
import numpy as np

# Set the device to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define the directory where the data is stored
# Replace with the path to your dataset
data_dir = '/Users/mrinoyb2/git/Wound-Detection/Wound_dataset copy'

# Define transforms for the data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomVerticalFlip(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset with ImageFolder
dataset = datasets.ImageFolder(data_dir, transform=transform)

#with wandb.init(project='wound-detection'):
# Split the dataset into train and validation sets
train_size = int(0.8 * len(dataset))  # 80% of dataset
val_size = len(dataset) - train_size  # 20% of dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create dataloaders
trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

# Load a pretrained resnet50 model
model = models.resnet50(pretrained=True)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False

# Replace the last fully connected layer with a layer tailored to the number of classes in your dataset
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(dataset.classes))

# Move the model to the GPU if available
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

# Train the model
epochs = 25
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # log results
        #wandb.log({'epoch': epoch, 'training_loss': loss})

    # Validation pass
    model.eval()  # Set model to evaluation mode
    val_loss = 0
    accuracy = 0
    with torch.no_grad():
        for inputs, labels in valloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            batch_loss = criterion(outputs, labels)
            val_loss += batch_loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct = (predicted == labels).type(torch.FloatTensor)
            accuracy += correct.mean()
            #wandb.log({'epoch': epoch, 'validation_loss': val_loss, 'accuracy': accuracy})

    print(f"Epoch {epoch+1}/{epochs}.. "
        f"Train loss: {running_loss/len(trainloader):.3f}.. "
        f"Validation loss: {val_loss/len(valloader):.3f}.. "
        f"Validation accuracy: {accuracy/len(valloader):.3f}")

# Save the model if you need to use it later
torch.save(model.state_dict(), '/Users/mrinoyb2/git/Wound-Detection/Models/wound_model_resnet34_25_epochs.pth')
#wandb.save('wound_model_1.pth')


/Users/mrinoyb2/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/mrinoyb2/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /Users/mrinoyb2/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100.0%


Epoch 1/25.. Train loss: 1.594.. Validation loss: 1.184.. Validation accuracy: 0.609
Epoch 2/25.. Train loss: 1.085.. Validation loss: 0.961.. Validation accuracy: 0.689
Epoch 3/25.. Train loss: 0.964.. Validation loss: 0.873.. Validation accuracy: 0.691
Epoch 4/25.. Train loss: 0.849.. Validation loss: 0.859.. Validation accuracy: 0.712
Epoch 5/25.. Train loss: 0.852.. Validation loss: 0.860.. Validation accuracy: 0.681
Epoch 6/25.. Train loss: 0.759.. Validation loss: 0.883.. Validation accuracy: 0.686
Epoch 7/25.. Train loss: 0.785.. Validation loss: 0.741.. Validation accuracy: 0.752
Epoch 8/25.. Train loss: 0.765.. Validation loss: 0.805.. Validation accuracy: 0.711
Epoch 9/25.. Train loss: 0.764.. Validation loss: 0.779.. Validation accuracy: 0.723
Epoch 10/25.. Train loss: 0.768.. Validation loss: 0.817.. Validation accuracy: 0.688
Epoch 11/25.. Train loss: 0.697.. Validation loss: 0.725.. Validation accuracy: 0.748
Epoch 12/25.. Train loss: 0.692.. Validation loss: 0.816.. Vali

## Using One cycle policy

In [15]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
import numpy as np

# Set the device to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#with wandb.init(project='wound-detection'):

# Define the directory where the data is stored
# Replace with the path to your dataset
data_dir = '/Users/mrinoybanerjee/Desktop/Duke/Classes/Spring 2024/AIPI 540 - Deep Learning/Wound-Detection/Data/Wound_dataset copy'

# Define transforms for the data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset with ImageFolder
dataset = datasets.ImageFolder(data_dir, transform=transform)

# Split the dataset into train and validation sets
train_size = int(0.8 * len(dataset))  # 80% of dataset
val_size = len(dataset) - train_size  # 20% of dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create dataloaders
trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

# Load a pretrained resnet50 model
model = models.resnet50(pretrained=True)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False

# Replace the last fully connected layer with a layer tailored to the number of classes in your dataset
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(dataset.classes))

# Move the model to the GPU if available
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
# use one cycle learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.1, steps_per_epoch=len(trainloader), epochs=10)

# Train the model
epochs = 10
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # log results
        #wandb.log({'epoch': epoch, 'training_loss': loss})

    # Validation pass
    model.eval()  # Set model to evaluation mode
    val_loss = 0
    accuracy = 0
    with torch.no_grad():
        for inputs, labels in valloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            batch_loss = criterion(outputs, labels)
            val_loss += batch_loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct = (predicted == labels).type(torch.FloatTensor)
            accuracy += correct.mean()
            #wandb.log({'epoch': epoch, 'validation_loss': val_loss, 'accuracy': accuracy})

    print(f"Epoch {epoch+1}/{epochs}.. "
        f"Train loss: {running_loss/len(trainloader):.3f}.. "
        f"Validation loss: {val_loss/len(valloader):.3f}.. "
        f"Validation accuracy: {accuracy/len(valloader):.3f}")
    
# Save the model if you need to use it later
torch.save(model.state_dict(), './Models/wound_model.pth')
#wandb.save('wound_model_1.pth')


/Users/mrinoybanerjee/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/mrinoybanerjee/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10.. Train loss: 1.777.. Validation loss: 1.556.. Validation accuracy: 0.529


KeyboardInterrupt: 

## Using inception model

In [11]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
import numpy as np

# Set the device to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define the directory where the data is stored
data_dir = '/Users/mrinoyb2/git/Wound-Detection/Wound_dataset copy'

# Define transforms for the data
# Update the resize transform for Inception's expected input size
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Adjusted for Inception
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomVerticalFlip(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset with ImageFolder
dataset = datasets.ImageFolder(data_dir, transform=transform)

# Split the dataset into train and validation sets
train_size = int(0.8 * len(dataset))  # 80% of dataset
val_size = len(dataset) - train_size  # 20% of dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create dataloaders
trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

# Load a pretrained Inception model
model = models.inception_v3(pretrained=True)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False

# Replace the last fully connected layer with a layer tailored to the number of classes in your dataset
# Inception uses model.fc for the final fully connected layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(dataset.classes))

# Move the model to the GPU if available
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

# Train the model
epochs = 25
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        # Handle the aux_logits for Inception model
        outputs, aux_outputs = model(inputs)
        loss1 = criterion(outputs, labels)
        loss2 = criterion(aux_outputs, labels)
        loss = loss1 + 0.4*loss2  # Combine losses

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation pass
    model.eval()  # Set model to evaluation mode
    val_loss = 0
    accuracy = 0
    with torch.no_grad():
        for inputs, labels in valloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)[0]  # Only use the main output for validation
            batch_loss = criterion(outputs, labels)
            val_loss += batch_loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct = (predicted == labels).type(torch.FloatTensor)
            accuracy += correct.mean()

    print(f"Epoch {epoch+1}/{epochs}.. "
        f"Train loss: {running_loss/len(trainloader):.3f}.. "
        f"Validation loss: {val_loss/len(valloader):.3f}.. "
        f"Validation accuracy: {accuracy/len(valloader):.3f}")

# Save the model if you need to use it later
torch.save(model.state_dict(), '/Users/mrinoyb2/git/Wound-Detection/Models/wound_model_inception_v3_25_epochs.pth')


/Users/mrinoyb2/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /Users/mrinoyb2/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100.0%


RuntimeError: size mismatch (got input: [10], target: [32])

## Model with Wandb logging

In [3]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
import wandb


# Initialize Weights & Biases
wandb.init(project='wound-detection')

def preprocess(data_dir):
    # Define transforms for the data
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.RandomVerticalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Load the dataset with ImageFolder
    dataset = datasets.ImageFolder(data_dir, transform=transform)

    # Split the dataset into train and validation sets
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    # Create dataloaders
    trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    valloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

    return trainloader, valloader, len(dataset.classes)

def train(model, device, trainloader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        model.train()
        running_loss = 0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        wandb.log({'epoch': epoch, 'training_loss': running_loss/len(trainloader)})

def validate(model, device, valloader, criterion):
    model.eval()
    val_loss = 0
    accuracy = 0
    with torch.no_grad():
        for inputs, labels in valloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            batch_loss = criterion(outputs, labels)
            val_loss += batch_loss.item()

            _, predicted = torch.max(outputs, 1)
            correct = (predicted == labels).type(torch.FloatTensor)
            accuracy += correct.mean()

    wandb.log({'validation_loss': val_loss/len(valloader), 'accuracy': accuracy/len(valloader)})

# Set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load data and preprocess
data_dir = '/Users/mrinoyb2/git/Wound-Detection/Wound_dataset copy'
trainloader, valloader, num_classes = preprocess(data_dir)

# Model setup
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

# Training and validation
epochs = 25
train(model, device, trainloader, criterion, optimizer, epochs)
validate(model, device, valloader, criterion)

# Save the model
torch.save(model.state_dict(), '/Users/mrinoyb2/git/Wound-Detection/Models/wound_model_wandb_25_epochs.pth')
wandb.save('wound_model_wandb_25_epochs.pth')

wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


/Users/mrinoyb2/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/mrinoyb2/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


accuracy,▁
epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
training_loss,█▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁
validation_loss,▁
accuracy,0.76042
epoch,24
training_loss,0.54755
validation_loss,0.7268
